# A Search and Download Test

In this tutorial we will download the command line client, demonstrate searching, and test our abilities (and credentials) to download data for later processing.

The SSARA (Seamless SAR Archive) federated query tool (ssara_federated_query.py) is a tool written by the EarthScope staff (and before them, the UNAVCO staff) to interrogate the various different public domain archives of SAR data. It is located on the SSARA GitHub page: https://gitlab.com/earthscope/public/sar/ssara_client.  You can clone the entire project and use git for tracking and pulling changes that have been made to the software.  

Documentation and API details are on the GitHub wiki page:  
https://gitlab.com/earthscope/public/sar/ssara_client/-/wikis/SSARA-API  
https://gitlab.com/earthscope/public/sar/ssara_client/-/wikis/Usage-Examples  

## 1. Download the SSARA Federated Query tool

In [ ]:
# First, make some directories to save it in, and go there

import os

wkdir=os.path.join(os.getenv("HOME"),"work")
if os.path.isdir(wkdir):
    os.chdir(wkdir)
else:
    os.mkdir(wkdir)
    os.chdir(wkdir)


In [ ]:
!git clone https://gitlab.com/earthscope/public/sar/ssara_client.git $HOME/work/SSARA

## 2. Using the SSARA Federated Query tool
Running the client without any options will print the help message and describe the options.  Some usage examples are given in the help as well.  Take some time to perform some example searches in a terminal window to get a feel from how the client works.

In [ ]:
!$HOME/work/SSARA/ssara_federated_query.py

## 3. Searching for Sentinel-1 data for the Ridgecrest earthquakes
Now that the client is working properly on our system, we will do a search for Sentinel-1 data covering the July 4 and 5, 2019 Ridgecrest earthquakes.  We want Sentinel-1, track (or relativeOrbit) 64, and to search for about 12 days around the earthquakes, in a small lat-long box around the city of Ridgecrest (35.6205° N, -117.6718° E):

In [ ]:
!$HOME/work/SSARA/ssara_federated_query.py --platform=SENTINEL-1 --relativeOrbit=64 --start=2019-07-01 --end=2019-07-13 -b '-117.9, -117.3, 35.3, 35.9' --print

The --print option prints the results in comma separated value (CSV) format to the screen with the following fields:  
collection,platform,start,end,relativeOrbit,first_frame,final_frame,beamMode,beamSwath,
flightDirection,lookDirection,polarization,downloadURL  

Now let's see what we can do with these results!

## 4. Test your download credentials

### 4.1 Can you download Sentinel-1 data at all?

You will see that there are clickable links returned for data filenames from your search (the 'downloadURL's from the csv list returned by SSARA). We can use these to test your download credentials. 

Try clicking on the last of those (the URL for the Sentinel-1B file). What happened? There are a few options:

1. You have an active EarthData account, with all license agreements made, and are logged into it. The data file will start downloading to your local machine through your web browser! (You should probably stop it, as it is a large file.)
2. You have an activate EarthData account, are logged into it, but have not accepted the license agreement. You will be taken to a web page where you can accept the license agreement, and when you do, the data should start to download locally, which you should probably stop (see 1. above).
3. You have not logged into your EarthData account, so will be taken to the EarthData login screen. After you log in, you will either be taken to a license screen (see 2. above), or start downloading the file (see 1. above).
4. You haven't made an EarthData account. Nothing is going to download. Go back to the Data Access Accounts notebook, follow the instructions there, and come back when you have made an EarthData account.

(**NOTE: there is no point in allowing the download to complete to your local machine, you are not going to process the data on your machine. This is just a test of whether your credentials work!**)

### 4.2 Is your `.netrc` file set up correctly?

If you are able to initiate a download by clicking on the links returned by SSARA, then you have correctly set up your account and given yourself the appropriate permissions. But that is not enough for success in the course! You have to have correctly written them to a file in your home directory (`.netrc`) in order for our codes to access and use them!

We can test whether you did that by trying to use them to download a file to your OpenScienceLab machine. We can use the command line `wget` command to do this. Simply give `wget` the URL of a downloadable file, e.g. like this:

In [ ]:
!wget https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20190710T014959_20190710T015026_017065_0201B8_069B.zip

What happens? Possibly one of two things:

1. You successfully download a Sentinel-1 SAR SLC file to your `work` directory (`/home/jovyan/work`). Congratulations, everything is set up properly!
2. You don't download anything. `wget` grumpily announces: "`Username/Password Authentication Failed.`" This means you have failed to set up your `.netrc` file properly. If you don't know what that means, you really should go back and run (and read, carefully) the Data_Access_Accounts notebook.

You can double check your `.netrc` even exists, by running this:

In [ ]:
!cat ~/.netrc

If this doesn't report your own unique login details for your NASA EarthData account, then you should go back and try the relevant section of Data_Access_Accounts again. **You will not be able to run any InSAR processing workflows until this is fixed!**

### 4.3 Do my OpenTopography credentials work?

You may have also noticed that SSARA can report the details of a digital elevation model (DEM) that covers your area of interest. You may also notice, that if you click on *that* URL, you don't get very far. (If you haven't tried it yet, try it now.) 

Do you get something like `Error: API Key required for access. Please register for an API key at www.opentopography.org`? I do, and I have an account on OpenTopography, and made a `.topoapi` file containing my API key and everything. If you did that too, then running this should reveal a long string of characters, made up of numbers and the letters a-f:

In [ ]:
!cat ~/.topoapi

If that is not what you get, then (guess what!) you should consider going back and trying the relevant section of the Data_Access_Accounts notebook again.

If, on the other hand, you have the correct key set up, well it's not your fault $-$ you need to add something to your download request.

The line of the SSARA response corresponding to the DEM download is written as a command line `curl` command $-$ another type of download command. If you copy and paste it into a code cell (and put an exclamation point in front of it), you would get this:

In [ ]:
!curl -X GET "https://portal.opentopography.org/API/globaldem?north=37.566679&south=33.697595&east=-115.955171&west=-119.703413&outputFormat=GTiff&demtype=SRTMGL1_E" -H "accept: */*" -o dem.wgs84.tif

If this command completes in a second or two and only downloads 162 bytes, it didn't work, and only downloaded an error message. You can confirm here:

In [ ]:
!grep required dem.wgs84.tif

If you see the same error message as you got before, then you know that it is not reading your API key. What you have to do is attach it as part of the long URL in quotation marks in the `curl` command. The exact text string you need to add should look like this:

In [ ]:
!cat ~/.topoapi | awk '{print "&API_Key="$1}'

And you can see an example in the code cell below. Be sure to edit the code cell to replace `&API_Key=ReplaceThisWithYourKey` with whatever was output by the previous code cell. Hopefully this will allow you to download the correct file, and not another error message... 

In [ ]:
!curl -X GET "https://portal.opentopography.org/API/globaldem?north=37.566679&south=33.697595&east=-115.955171&west=-119.703413&outputFormat=GTiff&demtype=SRTMGL1_E&API_Key=ReplaceThisWithYourKey" -H "accept: */*" -o dem.wgs84.tif

If you download a 578M file, then everything is working as it should. Hurrah!